In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import networkx as nx

import rpy2 as rp
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
import pyreadr

import os
import time
import h5py
# import mmdet
import scanpy
# import ssam
import pickle
import random
# import pyarrow
import dgl
import ast

from tqdm import tqdm

from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, accuracy_score, matthews_corrcoef, mean_squared_error

# from captum.attr import IntegratedGradients
# from captum.attr import LayerConductance
# from captum.attr import NeuronConductance

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.nn.functional as F

# import torch_geometric

from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import transforms, utils

from skimage import io
from skimage import img_as_float
from scipy import ndimage

# from ssam.utils import corr, calc_ctmap, calc_corrmap, flood_fill, calc_kde
from scipy.spatial.distance import cdist

from collections import OrderedDict

from packaging import version
import subprocess

In [2]:
import sys
print(sys.executable)

/opt/conda/bin/python


In [3]:
import anndata2ri

In [4]:
import scanpy as sc

In [5]:
# Activate the anndata2ri conversion between SingleCellExperiment and AnnData
anndata2ri.activate()

#Loading the rpy2 extension enables cell magic to be used
#This runs R code in jupyter notebook cells
%load_ext rpy2.ipython

sc.settings.verbosity = 3
sc.logging.print_versions()

-----
anndata     0.9.1
scanpy      1.9.3
-----
PIL                         9.4.0
anndata2ri                  1.2
anyio                       NA
arrow                       1.2.3
asciitree                   NA
asttokens                   NA
astunparse                  1.6.3
attr                        23.1.0
babel                       2.12.1
backcall                    0.2.0
bottleneck                  1.3.5
brotli                      NA
certifi                     2023.07.22
cffi                        1.15.1
chardet                     4.0.0
charset_normalizer          2.0.4
cloudpickle                 2.2.1
comm                        0.1.3
cycler                      0.10.0
cython_runtime              NA
dask                        2023.6.0
dateutil                    2.8.2
debugpy                     1.6.7
decorator                   5.1.1
defusedxml                  0.7.1
dgl                         1.1.0
entrypoints                 0.4
executing                   0.8.3
fastene

In [6]:
%%R
suppressPackageStartupMessages(library(Seurat))

# Load PBMC dataset from Seurat tutorial
pbmc.data <- Read10X(data.dir = "/workspace/spatial_integration/dataset/GraphST/mouse_anterior_section1/filtered_feature_bc_matrix/")
# Initialize the Seurat object with the raw (non-normalized data).
pbmc <- CreateSeuratObject(counts = pbmc.data, project = "mouse_ante_1")

pbmc


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    An object of class Seurat 
32285 features across 2695 samples within 1 assay 
Active assay: RNA (32285 features, 0 variable features)


R[write to console]: Error in dev.off() : cannot shut down device 1 (the null device)



RRuntimeError: Error in dev.off() : cannot shut down device 1 (the null device)


In [2]:
# df_a = pd.DataFrame([[1, 2, 3], [4, 5, 6]])
# df_b = pd.DataFrame([[7, 8, 9], [10, 11, 12]])

# df_c = df_a.T.dot(df_b)
# df_d = df_a.dot(df_b.T)
# df_c_ = df_b.T.dot(df_a)
# df_d_ = df_b.dot(df_a.T)

# np_a = df_a.to_numpy()
# np_b = df_b.to_numpy()

# df_c = np.matmul(np_a.T,np_b)
# df_d = np.matmul(np_a,np_b.T)
# df_c_ = np.matmul(np_b.T,np_a)
# df_d_ = np.matmul(np_b,np_a.T)

<h1>Take Matrix

In [3]:
# read from rds
mat_151507 = pyreadr.read_r('dataset/df_logcount_507.rds')
mat_151672 = pyreadr.read_r('dataset/df_logcount_672.rds')
mat_151673 = pyreadr.read_r('dataset/df_logcount_673.rds')
spat_507 = np.load('dataset/datapoints/com3_encoded_507_unfil.npy') 
spat_672 = np.load('dataset/datapoints/com3_encoded_672_unfil.npy') 
spat_673 = np.load('dataset/datapoints/com3_encoded_673_unfil.npy') 

In [4]:
spat_507.shape

(33538, 1, 4226)

In [5]:
spat_507.shape

(33538, 1, 4226)

In [6]:
# dict to dataframe
df_151507 = mat_151507[None]
df_151672 = mat_151672[None]
df_151673 = mat_151673[None]
combined_507_672 = pd.concat([df_151507,df_151672],axis=1)
combined_all = pd.concat([combined_507_672,df_151673],axis=1)

In [7]:
df_spat507 = pd.DataFrame(spat_507.reshape((spat_507.shape[0],spat_507.shape[-1])))
df_spat672 = pd.DataFrame(spat_672.reshape((spat_672.shape[0],spat_672.shape[-1])))
df_spat673 = pd.DataFrame(spat_673.reshape((spat_673.shape[0],spat_673.shape[-1])))

In [8]:
df_spat507.shape

(33538, 4226)

In [9]:
df_spat507

,0,1,2,3,4,5,6,7,8,9,...,4216,4217,4218,4219,4220,4221,4222,4223,4224,4225
0,0.500244,0.506533,0.502593,0.499145,0.501158,0.497553,0.507267,0.502238,0.499594,0.503867,...,0.501633,0.507672,0.505354,0.495214,0.502637,0.501169,0.503781,0.503091,0.505674,0.505318
1,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.501635,0.507704,0.505361,0.495214,0.502657,0.501170,0.503782,0.503083,0.505688,0.505336
2,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.501635,0.507704,0.505361,0.495214,0.502657,0.501170,0.503782,0.503083,0.505688,0.505336
3,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.501635,0.507704,0.505361,0.495214,0.502657,0.501170,0.503782,0.503083,0.505688,0.505336
4,0.500236,0.506519,0.502590,0.499143,0.501131,0.497548,0.507257,0.502214,0.499583,0.503840,...,0.501625,0.507652,0.505362,0.495220,0.502615,0.501176,0.503793,0.503103,0.505656,0.505299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33533,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.501635,0.507704,0.505361,0.495214,0.502657,0.501170,0.503782,0.503083,0.505688,0.505336
33534,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.501635,0.507704,0.505361,0.495214,0.502657,0.501170,0.503782,0.503083,0.505688,0.505336
33535,0.500241,0.506544,0.502597,0.499145,0.501166,0.497552,0.507275,0.502239,0.499593,0.503868,...,0.501632,0.507686,0.505359,0.495210,0.502644,0.501169,0.503782,0.503089,0.505682,0.505329
33536,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.501635,0.507704,0.505361,0.495214,0.502657,0.501170,0.503782,0.503083,0.505688,0.505336


In [10]:
tots = df_151507.shape[1] + df_151672.shape[1] + df_151673.shape[1]
tots

11880

In [11]:
df_spat507.columns = df_151507.columns
df_spat672.columns = df_151672.columns
df_spat673.columns = df_151673.columns

In [12]:
df_spat507

,AAACAACGAATAGTTC-1,AAACAAGTATCTCCCA-1,AAACAATCTACTAGCA-1,AAACACCAATAACTGC-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACAGTGTTCCTGGG-1,AAACATTTCCCGGATT-1,AAACCACTACACAGAT-1,AAACCCGAACGAAATC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTCTAGATACGCT-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTGTAAATTC-1
0,0.500244,0.506533,0.502593,0.499145,0.501158,0.497553,0.507267,0.502238,0.499594,0.503867,...,0.501633,0.507672,0.505354,0.495214,0.502637,0.501169,0.503781,0.503091,0.505674,0.505318
1,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.501635,0.507704,0.505361,0.495214,0.502657,0.501170,0.503782,0.503083,0.505688,0.505336
2,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.501635,0.507704,0.505361,0.495214,0.502657,0.501170,0.503782,0.503083,0.505688,0.505336
3,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.501635,0.507704,0.505361,0.495214,0.502657,0.501170,0.503782,0.503083,0.505688,0.505336
4,0.500236,0.506519,0.502590,0.499143,0.501131,0.497548,0.507257,0.502214,0.499583,0.503840,...,0.501625,0.507652,0.505362,0.495220,0.502615,0.501176,0.503793,0.503103,0.505656,0.505299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33533,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.501635,0.507704,0.505361,0.495214,0.502657,0.501170,0.503782,0.503083,0.505688,0.505336
33534,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.501635,0.507704,0.505361,0.495214,0.502657,0.501170,0.503782,0.503083,0.505688,0.505336
33535,0.500241,0.506544,0.502597,0.499145,0.501166,0.497552,0.507275,0.502239,0.499593,0.503868,...,0.501632,0.507686,0.505359,0.495210,0.502644,0.501169,0.503782,0.503089,0.505682,0.505329
33536,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.501635,0.507704,0.505361,0.495214,0.502657,0.501170,0.503782,0.503083,0.505688,0.505336


In [13]:
df_spat672

,AAACAAGTATCTCCCA-1,AAACACCAATAACTGC-1,AAACAGAGCGACTCCT-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACATTTCCCGGATT-1,AAACCACTACACAGAT-1,AAACCCGAACGAAATC-1,AAACCGGAAATGTTAA-1,AAACCGGGTAGGTACC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTCTAGATACGCT-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTGTAAATTC-1
0,0.501554,0.503273,0.502111,0.505629,0.503639,0.502576,0.502582,0.501222,0.501362,0.504176,...,0.499378,0.497233,0.506778,0.501806,0.504006,0.505121,0.497932,0.505613,0.497251,0.502621
1,0.501557,0.503285,0.502129,0.505632,0.503639,0.502582,0.502596,0.501222,0.501376,0.504186,...,0.499388,0.497234,0.506799,0.501813,0.504009,0.505124,0.497935,0.505632,0.497253,0.502628
2,0.501557,0.503285,0.502129,0.505632,0.503639,0.502582,0.502596,0.501222,0.501376,0.504186,...,0.499388,0.497234,0.506799,0.501813,0.504009,0.505124,0.497935,0.505632,0.497253,0.502628
3,0.501557,0.503285,0.502129,0.505632,0.503639,0.502582,0.502596,0.501222,0.501376,0.504186,...,0.499388,0.497234,0.506799,0.501813,0.504009,0.505124,0.497935,0.505632,0.497253,0.502628
4,0.501536,0.503233,0.502085,0.505620,0.503626,0.502561,0.502547,0.501202,0.501335,0.504144,...,0.499362,0.497232,0.506745,0.501781,0.503989,0.505110,0.497906,0.505584,0.497256,0.502595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33533,0.501557,0.503285,0.502129,0.505632,0.503639,0.502582,0.502596,0.501222,0.501376,0.504186,...,0.499388,0.497234,0.506799,0.501813,0.504009,0.505124,0.497935,0.505632,0.497253,0.502628
33534,0.501557,0.503285,0.502129,0.505632,0.503639,0.502582,0.502596,0.501222,0.501376,0.504186,...,0.499388,0.497234,0.506799,0.501813,0.504009,0.505124,0.497935,0.505632,0.497253,0.502628
33535,0.501550,0.503267,0.502105,0.505622,0.503628,0.502581,0.502578,0.501216,0.501358,0.504162,...,0.499370,0.497240,0.506767,0.501799,0.504004,0.505112,0.497926,0.505605,0.497247,0.502614
33536,0.501557,0.503285,0.502129,0.505632,0.503639,0.502582,0.502596,0.501222,0.501376,0.504186,...,0.499388,0.497234,0.506799,0.501813,0.504009,0.505124,0.497935,0.505632,0.497253,0.502628


In [14]:
df_spat673

,AAACAAGTATCTCCCA-1,AAACAATCTACTAGCA-1,AAACACCAATAACTGC-1,AAACAGAGCGACTCCT-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACAGTGTTCCTGGG-1,AAACATTTCCCGGATT-1,AAACCCGAACGAAATC-1,AAACCGGGTAGGTACC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTATTACACG-1,TTGTTTGTGTAAATTC-1
0,0.499335,0.502461,0.506334,0.501288,0.506756,0.507025,0.503188,0.506514,0.502352,0.500726,...,0.499240,0.502114,0.505924,0.503782,0.499508,0.507652,0.499881,0.504928,0.508541,0.496654
1,0.499350,0.502478,0.506350,0.501297,0.506780,0.507063,0.503211,0.506526,0.502355,0.500736,...,0.499243,0.502132,0.505944,0.503789,0.499511,0.507681,0.499889,0.504931,0.508569,0.496651
2,0.499350,0.502478,0.506350,0.501297,0.506780,0.507063,0.503211,0.506526,0.502355,0.500736,...,0.499243,0.502132,0.505944,0.503789,0.499511,0.507681,0.499889,0.504931,0.508569,0.496651
3,0.499350,0.502478,0.506350,0.501297,0.506780,0.507063,0.503211,0.506526,0.502355,0.500736,...,0.499243,0.502132,0.505944,0.503789,0.499511,0.507681,0.499889,0.504931,0.508569,0.496651
4,0.499302,0.502415,0.506299,0.501233,0.506690,0.506920,0.503121,0.506472,0.502353,0.500678,...,0.499206,0.502057,0.505879,0.503759,0.499490,0.507596,0.499867,0.504900,0.508458,0.496664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33533,0.499350,0.502478,0.506350,0.501297,0.506780,0.507063,0.503211,0.506526,0.502355,0.500736,...,0.499243,0.502132,0.505944,0.503789,0.499511,0.507681,0.499889,0.504931,0.508569,0.496651
33534,0.499350,0.502478,0.506350,0.501297,0.506780,0.507063,0.503211,0.506526,0.502355,0.500736,...,0.499243,0.502132,0.505944,0.503789,0.499511,0.507681,0.499889,0.504931,0.508569,0.496651
33535,0.499322,0.502441,0.506316,0.501272,0.506731,0.506981,0.503160,0.506505,0.502350,0.500716,...,0.499235,0.502098,0.505894,0.503776,0.499501,0.507628,0.499864,0.504907,0.508502,0.496662
33536,0.499350,0.502478,0.506350,0.501297,0.506780,0.507063,0.503211,0.506526,0.502355,0.500736,...,0.499243,0.502132,0.505944,0.503789,0.499511,0.507681,0.499889,0.504931,0.508569,0.496651


In [15]:
ind_list = df_151507.index
new_list = []
for i in range(len(ind_list)):
    # new_list.append(ind_list[i]+'-spat')
    new_list.append(ind_list[i])
df_spat507.index = new_list
df_spat672.index = new_list
df_spat673.index = new_list

In [16]:
df_151507

,AAACAACGAATAGTTC-1,AAACAAGTATCTCCCA-1,AAACAATCTACTAGCA-1,AAACACCAATAACTGC-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACAGTGTTCCTGGG-1,AAACATTTCCCGGATT-1,AAACCACTACACAGAT-1,AAACCCGAACGAAATC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTCTAGATACGCT-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTGTAAATTC-1
rownames,,,,,,,,,,,,,,,,,,,,,
ENSG00000243485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000237613,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000186092,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000238009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000239945,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000277856,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000275063,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000271254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
df_151672

,AAACAAGTATCTCCCA-1,AAACACCAATAACTGC-1,AAACAGAGCGACTCCT-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACATTTCCCGGATT-1,AAACCACTACACAGAT-1,AAACCCGAACGAAATC-1,AAACCGGAAATGTTAA-1,AAACCGGGTAGGTACC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTCTAGATACGCT-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTGTAAATTC-1
rownames,,,,,,,,,,,,,,,,,,,,,
ENSG00000243485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000237613,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000186092,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000238009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000239945,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000277856,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000275063,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000271254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
df_151673

,AAACAAGTATCTCCCA-1,AAACAATCTACTAGCA-1,AAACACCAATAACTGC-1,AAACAGAGCGACTCCT-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACAGTGTTCCTGGG-1,AAACATTTCCCGGATT-1,AAACCCGAACGAAATC-1,AAACCGGGTAGGTACC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTATTACACG-1,TTGTTTGTGTAAATTC-1
rownames,,,,,,,,,,,,,,,,,,,,,
ENSG00000243485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000237613,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000186092,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000238009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000239945,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000277856,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000275063,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000271254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
embed_507 = df_151507.add(df_spat507)
embed_507

,AAACAACGAATAGTTC-1,AAACAAGTATCTCCCA-1,AAACAATCTACTAGCA-1,AAACACCAATAACTGC-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACAGTGTTCCTGGG-1,AAACATTTCCCGGATT-1,AAACCACTACACAGAT-1,AAACCCGAACGAAATC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTCTAGATACGCT-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTGTAAATTC-1
rownames,,,,,,,,,,,,,,,,,,,,,
ENSG00000243485,0.500244,0.506533,0.502593,0.499145,0.501158,0.497553,0.507267,0.502238,0.499594,0.503867,...,0.501633,0.507672,0.505354,0.495214,0.502637,0.501169,0.503781,0.503091,0.505674,0.505318
ENSG00000237613,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.501635,0.507704,0.505361,0.495214,0.502657,0.501170,0.503782,0.503083,0.505688,0.505336
ENSG00000186092,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.501635,0.507704,0.505361,0.495214,0.502657,0.501170,0.503782,0.503083,0.505688,0.505336
ENSG00000238009,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.501635,0.507704,0.505361,0.495214,0.502657,0.501170,0.503782,0.503083,0.505688,0.505336
ENSG00000239945,0.500236,0.506519,0.502590,0.499143,0.501131,0.497548,0.507257,0.502214,0.499583,0.503840,...,0.501625,0.507652,0.505362,0.495220,0.502615,0.501176,0.503793,0.503103,0.505656,0.505299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000277856,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.501635,0.507704,0.505361,0.495214,0.502657,0.501170,0.503782,0.503083,0.505688,0.505336
ENSG00000275063,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.501635,0.507704,0.505361,0.495214,0.502657,0.501170,0.503782,0.503083,0.505688,0.505336
ENSG00000271254,0.500241,0.506544,0.502597,0.499145,0.501166,0.497552,0.507275,0.502239,0.499593,0.503868,...,0.501632,0.507686,0.505359,0.495210,0.502644,0.501169,0.503782,0.503089,0.505682,0.505329


In [20]:
embed_672 = df_151672.add(df_spat672)
embed_672

,AAACAAGTATCTCCCA-1,AAACACCAATAACTGC-1,AAACAGAGCGACTCCT-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACATTTCCCGGATT-1,AAACCACTACACAGAT-1,AAACCCGAACGAAATC-1,AAACCGGAAATGTTAA-1,AAACCGGGTAGGTACC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTCTAGATACGCT-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTGTAAATTC-1
rownames,,,,,,,,,,,,,,,,,,,,,
ENSG00000243485,0.501554,0.503273,0.502111,0.505629,0.503639,0.502576,0.502582,0.501222,0.501362,0.504176,...,0.499378,0.497233,0.506778,0.501806,0.504006,0.505121,0.497932,0.505613,0.497251,0.502621
ENSG00000237613,0.501557,0.503285,0.502129,0.505632,0.503639,0.502582,0.502596,0.501222,0.501376,0.504186,...,0.499388,0.497234,0.506799,0.501813,0.504009,0.505124,0.497935,0.505632,0.497253,0.502628
ENSG00000186092,0.501557,0.503285,0.502129,0.505632,0.503639,0.502582,0.502596,0.501222,0.501376,0.504186,...,0.499388,0.497234,0.506799,0.501813,0.504009,0.505124,0.497935,0.505632,0.497253,0.502628
ENSG00000238009,0.501557,0.503285,0.502129,0.505632,0.503639,0.502582,0.502596,0.501222,0.501376,0.504186,...,0.499388,0.497234,0.506799,0.501813,0.504009,0.505124,0.497935,0.505632,0.497253,0.502628
ENSG00000239945,0.501536,0.503233,0.502085,0.505620,0.503626,0.502561,0.502547,0.501202,0.501335,0.504144,...,0.499362,0.497232,0.506745,0.501781,0.503989,0.505110,0.497906,0.505584,0.497256,0.502595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000277856,0.501557,0.503285,0.502129,0.505632,0.503639,0.502582,0.502596,0.501222,0.501376,0.504186,...,0.499388,0.497234,0.506799,0.501813,0.504009,0.505124,0.497935,0.505632,0.497253,0.502628
ENSG00000275063,0.501557,0.503285,0.502129,0.505632,0.503639,0.502582,0.502596,0.501222,0.501376,0.504186,...,0.499388,0.497234,0.506799,0.501813,0.504009,0.505124,0.497935,0.505632,0.497253,0.502628
ENSG00000271254,0.501550,0.503267,0.502105,0.505622,0.503628,0.502581,0.502578,0.501216,0.501358,0.504162,...,0.499370,0.497240,0.506767,0.501799,0.504004,0.505112,0.497926,0.505605,0.497247,0.502614


In [21]:
embed_673 = df_151673.add(df_spat673)
embed_673

,AAACAAGTATCTCCCA-1,AAACAATCTACTAGCA-1,AAACACCAATAACTGC-1,AAACAGAGCGACTCCT-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACAGTGTTCCTGGG-1,AAACATTTCCCGGATT-1,AAACCCGAACGAAATC-1,AAACCGGGTAGGTACC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTATTACACG-1,TTGTTTGTGTAAATTC-1
rownames,,,,,,,,,,,,,,,,,,,,,
ENSG00000243485,0.499335,0.502461,0.506334,0.501288,0.506756,0.507025,0.503188,0.506514,0.502352,0.500726,...,0.499240,0.502114,0.505924,0.503782,0.499508,0.507652,0.499881,0.504928,0.508541,0.496654
ENSG00000237613,0.499350,0.502478,0.506350,0.501297,0.506780,0.507063,0.503211,0.506526,0.502355,0.500736,...,0.499243,0.502132,0.505944,0.503789,0.499511,0.507681,0.499889,0.504931,0.508569,0.496651
ENSG00000186092,0.499350,0.502478,0.506350,0.501297,0.506780,0.507063,0.503211,0.506526,0.502355,0.500736,...,0.499243,0.502132,0.505944,0.503789,0.499511,0.507681,0.499889,0.504931,0.508569,0.496651
ENSG00000238009,0.499350,0.502478,0.506350,0.501297,0.506780,0.507063,0.503211,0.506526,0.502355,0.500736,...,0.499243,0.502132,0.505944,0.503789,0.499511,0.507681,0.499889,0.504931,0.508569,0.496651
ENSG00000239945,0.499302,0.502415,0.506299,0.501233,0.506690,0.506920,0.503121,0.506472,0.502353,0.500678,...,0.499206,0.502057,0.505879,0.503759,0.499490,0.507596,0.499867,0.504900,0.508458,0.496664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000277856,0.499350,0.502478,0.506350,0.501297,0.506780,0.507063,0.503211,0.506526,0.502355,0.500736,...,0.499243,0.502132,0.505944,0.503789,0.499511,0.507681,0.499889,0.504931,0.508569,0.496651
ENSG00000275063,0.499350,0.502478,0.506350,0.501297,0.506780,0.507063,0.503211,0.506526,0.502355,0.500736,...,0.499243,0.502132,0.505944,0.503789,0.499511,0.507681,0.499889,0.504931,0.508569,0.496651
ENSG00000271254,0.499322,0.502441,0.506316,0.501272,0.506731,0.506981,0.503160,0.506505,0.502350,0.500716,...,0.499235,0.502098,0.505894,0.503776,0.499501,0.507628,0.499864,0.504907,0.508502,0.496662


In [22]:
embed_507_672 = pd.concat([embed_507,embed_672],axis=1)
embed_507_672

,AAACAACGAATAGTTC-1,AAACAAGTATCTCCCA-1,AAACAATCTACTAGCA-1,AAACACCAATAACTGC-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACAGTGTTCCTGGG-1,AAACATTTCCCGGATT-1,AAACCACTACACAGAT-1,AAACCCGAACGAAATC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTCTAGATACGCT-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTGTAAATTC-1
rownames,,,,,,,,,,,,,,,,,,,,,
ENSG00000243485,0.500244,0.506533,0.502593,0.499145,0.501158,0.497553,0.507267,0.502238,0.499594,0.503867,...,0.499378,0.497233,0.506778,0.501806,0.504006,0.505121,0.497932,0.505613,0.497251,0.502621
ENSG00000237613,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.499388,0.497234,0.506799,0.501813,0.504009,0.505124,0.497935,0.505632,0.497253,0.502628
ENSG00000186092,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.499388,0.497234,0.506799,0.501813,0.504009,0.505124,0.497935,0.505632,0.497253,0.502628
ENSG00000238009,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.499388,0.497234,0.506799,0.501813,0.504009,0.505124,0.497935,0.505632,0.497253,0.502628
ENSG00000239945,0.500236,0.506519,0.502590,0.499143,0.501131,0.497548,0.507257,0.502214,0.499583,0.503840,...,0.499362,0.497232,0.506745,0.501781,0.503989,0.505110,0.497906,0.505584,0.497256,0.502595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000277856,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.499388,0.497234,0.506799,0.501813,0.504009,0.505124,0.497935,0.505632,0.497253,0.502628
ENSG00000275063,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.499388,0.497234,0.506799,0.501813,0.504009,0.505124,0.497935,0.505632,0.497253,0.502628
ENSG00000271254,0.500241,0.506544,0.502597,0.499145,0.501166,0.497552,0.507275,0.502239,0.499593,0.503868,...,0.499370,0.497240,0.506767,0.501799,0.504004,0.505112,0.497926,0.505605,0.497247,0.502614


In [23]:
embed_all = pd.concat([embed_507_672,embed_673],axis=1)
embed_all

,AAACAACGAATAGTTC-1,AAACAAGTATCTCCCA-1,AAACAATCTACTAGCA-1,AAACACCAATAACTGC-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACAGTGTTCCTGGG-1,AAACATTTCCCGGATT-1,AAACCACTACACAGAT-1,AAACCCGAACGAAATC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTATTACACG-1,TTGTTTGTGTAAATTC-1
rownames,,,,,,,,,,,,,,,,,,,,,
ENSG00000243485,0.500244,0.506533,0.502593,0.499145,0.501158,0.497553,0.507267,0.502238,0.499594,0.503867,...,0.499240,0.502114,0.505924,0.503782,0.499508,0.507652,0.499881,0.504928,0.508541,0.496654
ENSG00000237613,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.499243,0.502132,0.505944,0.503789,0.499511,0.507681,0.499889,0.504931,0.508569,0.496651
ENSG00000186092,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.499243,0.502132,0.505944,0.503789,0.499511,0.507681,0.499889,0.504931,0.508569,0.496651
ENSG00000238009,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.499243,0.502132,0.505944,0.503789,0.499511,0.507681,0.499889,0.504931,0.508569,0.496651
ENSG00000239945,0.500236,0.506519,0.502590,0.499143,0.501131,0.497548,0.507257,0.502214,0.499583,0.503840,...,0.499206,0.502057,0.505879,0.503759,0.499490,0.507596,0.499867,0.504900,0.508458,0.496664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000277856,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.499243,0.502132,0.505944,0.503789,0.499511,0.507681,0.499889,0.504931,0.508569,0.496651
ENSG00000275063,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.499243,0.502132,0.505944,0.503789,0.499511,0.507681,0.499889,0.504931,0.508569,0.496651
ENSG00000271254,0.500241,0.506544,0.502597,0.499145,0.501166,0.497552,0.507275,0.502239,0.499593,0.503868,...,0.499235,0.502098,0.505894,0.503776,0.499501,0.507628,0.499864,0.504907,0.508502,0.496662


In [24]:
embed_all.fillna(0,inplace=True)
embed_all

,AAACAACGAATAGTTC-1,AAACAAGTATCTCCCA-1,AAACAATCTACTAGCA-1,AAACACCAATAACTGC-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACAGTGTTCCTGGG-1,AAACATTTCCCGGATT-1,AAACCACTACACAGAT-1,AAACCCGAACGAAATC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTATTACACG-1,TTGTTTGTGTAAATTC-1
rownames,,,,,,,,,,,,,,,,,,,,,
ENSG00000243485,0.500244,0.506533,0.502593,0.499145,0.501158,0.497553,0.507267,0.502238,0.499594,0.503867,...,0.499240,0.502114,0.505924,0.503782,0.499508,0.507652,0.499881,0.504928,0.508541,0.496654
ENSG00000237613,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.499243,0.502132,0.505944,0.503789,0.499511,0.507681,0.499889,0.504931,0.508569,0.496651
ENSG00000186092,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.499243,0.502132,0.505944,0.503789,0.499511,0.507681,0.499889,0.504931,0.508569,0.496651
ENSG00000238009,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.499243,0.502132,0.505944,0.503789,0.499511,0.507681,0.499889,0.504931,0.508569,0.496651
ENSG00000239945,0.500236,0.506519,0.502590,0.499143,0.501131,0.497548,0.507257,0.502214,0.499583,0.503840,...,0.499206,0.502057,0.505879,0.503759,0.499490,0.507596,0.499867,0.504900,0.508458,0.496664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000277856,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.499243,0.502132,0.505944,0.503789,0.499511,0.507681,0.499889,0.504931,0.508569,0.496651
ENSG00000275063,0.500246,0.506548,0.502603,0.499145,0.501173,0.497557,0.507280,0.502253,0.499599,0.503876,...,0.499243,0.502132,0.505944,0.503789,0.499511,0.507681,0.499889,0.504931,0.508569,0.496651
ENSG00000271254,0.500241,0.506544,0.502597,0.499145,0.501166,0.497552,0.507275,0.502239,0.499593,0.503868,...,0.499235,0.502098,0.505894,0.503776,0.499501,0.507628,0.499864,0.504907,0.508502,0.496662


In [25]:
# embed_507_672 = pd.concat([embed_507,embed_672],axis=1)
# embed_all = pd.concat([embed_507_672,embed_673],axis=1)
# embed_all

In [43]:
# embed_all.to_csv('dataset/datapoints/embed_all.csv',index = False)

# Save DataFrame to CSV with progress bar
chunks = np.array_split(embed_all.index, 1000) # split into 1000 chunks

for chunck, subset in enumerate(tqdm(chunks)):
    if chunck == 0: # first row
        embed_all.loc[subset].to_csv('dataset/datapoints/embed_all_com4_add_unfil.csv', mode='w', index=True)
    else:
        embed_all.loc[subset].to_csv('dataset/datapoints/embed_all_com4_add_unfil.csv', header=None, mode='a', index=True)
# embed_all.to_csv('dataset/datapoints/embed_all_com3.csv',index = True, chunksize=1000)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [04:37<00:00,  3.60it/s]


In [26]:
print('saving...')
embed_all.to_pickle('dataset/datapoints/embed_all_com4_add_unfil.pkl')
print('done')

saving...
done


In [47]:
import sys
a = sys.getsizeof(df_spat507)/(1024 ** 2)
b = sys.getsizeof(df_151507)/(1024 ** 2)
c = sys.getsizeof(df_spat672)/(1024 ** 2)
d = sys.getsizeof(df_151672)/(1024 ** 2)
e = sys.getsizeof(df_spat673)/(1024 ** 2)
f = sys.getsizeof(df_151673)/(1024 ** 2)
print(a)
print(b)
print(c)
print(d)
print(e)
print(f)

print('total: ',a+b+c+d+e+f)
print(sys.getsizeof(embed_all)/(1024 ** 2))

1083.789011001587
1083.6290893554688
1029.7994632720947
1029.6395416259766
933.5906009674072
933.4306793212891
total:  6093.878385543823
6086.362642288208


In [42]:
import sys
a = sys.getsizeof(df_spat507)/(1024 ** 2)
b = sys.getsizeof(df_151507)/(1024 ** 2)
c = sys.getsizeof(df_spat672)/(1024 ** 2)
d = sys.getsizeof(df_151672)/(1024 ** 2)
e = sys.getsizeof(df_spat673)/(1024 ** 2)
f = sys.getsizeof(df_151673)/(1024 ** 2)
print(a)
print(b)
print(c)
print(d)
print(e)
print(f)

print('total: ',a+b+c+d+e+f)
print(sys.getsizeof(embed_all)/(1024 ** 2))

1083.6290893554688
1084.6369400024414
1029.6395416259766
1029.6395416259766
933.4306793212891
933.4306793212891
total:  6094.406471252441
3043.101387023926


In [23]:
# embed_507.to_csv('datapoints/embedded_507.csv', index=False)
pyreadr.write_rds('datapoints/embed_507.rds', embed_507, compress='gzip')

In [24]:
pyreadr.write_rds('datapoints/embed_672.rds', embed_672, compress='gzip')

In [25]:
pyreadr.write_rds('datapoints/embed_673.rds', embed_673, compress='gzip')

In [13]:
# filters gene with 0 value throughout assay matrix
summ = df_151507.sum(axis=1)
filtered_151507 = df_151507.loc[summ>0]
summ = df_151672.sum(axis=1)
filtered_151672 = df_151672.loc[summ>0]
summ = df_151673.sum(axis=1)
filtered_151673 = df_151673.loc[summ>0]
summ = combined_all.sum(axis=1)
filtered_all = combined_all.loc[summ>0]

# this results in a non-uniform rows, should concate first before filters

In [14]:
# selected gene after filtering non-zero across all samples
df = pd.DataFrame(filtered_all.index.tolist())
selected_gene = df[0].tolist()

In [17]:
len(selected_gene)

23465

In [15]:
df_151673.loc[selected_gene]

,AAACAAGTATCTCCCA-1,AAACAATCTACTAGCA-1,AAACACCAATAACTGC-1,AAACAGAGCGACTCCT-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACAGTGTTCCTGGG-1,AAACATTTCCCGGATT-1,AAACCCGAACGAAATC-1,AAACCGGGTAGGTACC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTATTACACG-1,TTGTTTGTGTAAATTC-1
rownames,,,,,,,,,,,,,,,,,,,,,
ENSG00000243485,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000238009,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000229905,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000237491,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000177757,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000278817,0.0,0.0,0.0,0.0,0.804885,0.0,0.0,0.481892,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000277196,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.328571,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000278384,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
df['coord_list'] = pd.Series(dtype=str)
df['coord_count'] = pd.Series(dtype=int)

In [20]:
df_507 = df.copy()
df_673 = df.copy()
df_672 = df.copy()

In [21]:
df

,0,coord_list,coord_count
0,ENSG00000243485,NaN,NaN
1,ENSG00000238009,NaN,NaN
2,ENSG00000229905,NaN,NaN
3,ENSG00000237491,NaN,NaN
4,ENSG00000177757,NaN,NaN
...,...,...,...
23460,ENSG00000278817,NaN,NaN
23461,ENSG00000277196,NaN,NaN
23462,ENSG00000278384,NaN,NaN
23463,ENSG00000271254,NaN,NaN


<h1>Preprocess

In [22]:
# assay into multiple instances

# read each spots coordinates
coord507 = pyreadr.read_r('spatialLIBD/r_lib/coord_151507.rds')
coord672 = pyreadr.read_r('spatialLIBD/r_lib/coord_151672.rds')
coord673 = pyreadr.read_r('spatialLIBD/r_lib/coord_151673.rds')

# coordinates in respect to gene count values

# output should be matrix of gene distribution for each gene

In [23]:
coord507 = coord507[None]
coord672 = coord672[None]
coord673 = coord673[None]

In [24]:
coord507

,pxl_col_in_fullres,pxl_row_in_fullres
AAACAACGAATAGTTC-1,3276.0,2514.0
AAACAAGTATCTCCCA-1,9178.0,8520.0
AAACAATCTACTAGCA-1,5133.0,2878.0
AAACACCAATAACTGC-1,3462.0,9581.0
AAACAGCTTTCAGAAG-1,2779.0,7663.0
...,...,...
TTGTTGTGTGTCAAGA-1,7464.0,6239.0
TTGTTTCACATCCAGG-1,5045.0,9466.0
TTGTTTCATTAGTCTA-1,4218.0,9703.0
TTGTTTCCATACAACT-1,4017.0,7906.0


In [530]:
coord672

,pxl_col_in_fullres,pxl_row_in_fullres
AAACAAGTATCTCCCA-1,10215.0,8703.0
AAACACCAATAACTGC-1,4499.0,9767.0
AAACAGAGCGACTCCT-1,9675.0,4390.0
AAACAGCTTTCAGAAG-1,3816.0,7849.0
AAACAGGGTCTATATT-1,4090.0,8329.0
...,...,...
TTGTTGTGTGTCAAGA-1,8500.0,6423.0
TTGTTTCACATCCAGG-1,6083.0,9651.0
TTGTTTCATTAGTCTA-1,5256.0,9889.0
TTGTTTCCATACAACT-1,5054.0,8092.0


In [431]:
coord673

,pxl_col_in_fullres,pxl_row_in_fullres
AAACAAGTATCTCCCA-1,9791.0,8468.0
AAACAATCTACTAGCA-1,5769.0,2807.0
AAACACCAATAACTGC-1,4068.0,9505.0
AAACAGAGCGACTCCT-1,9271.0,4151.0
AAACAGCTTTCAGAAG-1,3393.0,7583.0
...,...,...
TTGTTTCACATCCAGG-1,5653.0,9396.0
TTGTTTCATTAGTCTA-1,4825.0,9630.0
TTGTTTCCATACAACT-1,4631.0,7831.0
TTGTTTGTATTACACG-1,5571.0,11193.0


<h5> get coordinate list from assay

In [541]:
# get selected gene from individual sample
select_507 = df_151507.loc[selected_gene]
select_507

,AAACAACGAATAGTTC-1,AAACAAGTATCTCCCA-1,AAACAATCTACTAGCA-1,AAACACCAATAACTGC-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACAGTGTTCCTGGG-1,AAACATTTCCCGGATT-1,AAACCACTACACAGAT-1,AAACCCGAACGAAATC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTCTAGATACGCT-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTGTAAATTC-1
rownames,,,,,,,,,,,,,,,,,,,,,
ENSG00000243485,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000238009,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000229905,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000237491,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000177757,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000278817,0.0,0.841739,0.0,0.0,0.0,0.0,1.382027,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000277196,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.593077,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000278384,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [542]:
# fiters rows from assays, get coordinate list and coordinate count
for i in tqdm(range(len(filtered_all))):   
    a = select_507.iloc[i]
    df_507.loc[i, 'coord_list'] = str(a[a>0].index.tolist())
    df_507.loc[i, 'coord_count'] = len(a[a>0].index.tolist())

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23465/23465 [00:39<00:00, 597.50it/s]


In [543]:
# Number of gene that have non-zero values, from the initial 33538. should be used as the final shape
df

,0,coord_list,coord_count
0,ENSG00000243485,NaN,NaN
1,ENSG00000238009,NaN,NaN
2,ENSG00000229905,NaN,NaN
3,ENSG00000237491,NaN,NaN
4,ENSG00000177757,NaN,NaN
...,...,...,...
23460,ENSG00000278817,NaN,NaN
23461,ENSG00000277196,NaN,NaN
23462,ENSG00000278384,NaN,NaN
23463,ENSG00000271254,NaN,NaN


In [544]:
df_672

,0,coord_list,coord_count
0,ENSG00000243485,[],0.0
1,ENSG00000238009,"['ACCCTATAGGACTGAG-1', 'ATCGCCAGTCAACATT-1', '...",13.0
2,ENSG00000229905,['GCTAAACCTGAGGTGA-1'],1.0
3,ENSG00000237491,"['AACCAAGACTTCTCTG-1', 'AACCCTACTGTCAATA-1', '...",84.0
4,ENSG00000177757,[],0.0
...,...,...,...
23460,ENSG00000278817,"['AAACCGGAAATGTTAA-1', 'AAACGACAGTCTTGCC-1', '...",620.0
23461,ENSG00000277196,"['AAATTGGTGAGAAGCA-1', 'AAATTTGCGGGTGTGG-1', '...",65.0
23462,ENSG00000278384,"['CCCAACATACGTCGCG-1', 'TATTAACACCAAAGCA-1']",2.0
23463,ENSG00000271254,"['AATTGAACGCTCTGGT-1', 'AGACCATGGGATACAA-1', '...",7.0


In [545]:
df_507

,0,coord_list,coord_count
0,ENSG00000243485,['TTGCCAAGCAGAACCC-1'],1.0
1,ENSG00000238009,"['CGCAATTAGGGTAATA-1', 'CTGCAAGCACGTTCCG-1', '...",6.0
2,ENSG00000229905,[],0.0
3,ENSG00000237491,"['AAAGACCCAAGTCGCG-1', 'AAAGTAGCATTGCTCA-1', '...",57.0
4,ENSG00000177757,['GGAGGGTCAAGTAAGA-1'],1.0
...,...,...,...
23460,ENSG00000278817,"['AAACAAGTATCTCCCA-1', 'AAACAGTGTTCCTGGG-1', '...",530.0
23461,ENSG00000277196,"['AAACCACTACACAGAT-1', 'AACTGGGTTCGAGCCG-1', '...",44.0
23462,ENSG00000278384,"['AGTCACTCCGCCTCAT-1', 'ATACGGTGAAGATGCA-1']",2.0
23463,ENSG00000271254,"['AGAAGGTTGCCGAATT-1', 'AGACTGTTACCGGGTC-1', '...",6.0


<h3>Filter genes with minimum points of coordinates (currently 250)

In [546]:
# filter genes with minimum points of coordinates
df_507_min = df_507.loc[:][df_507.loc[:]['coord_count']>250]

In [547]:
df_672_min

,0,coord_list,coord_count
11,ENSG00000188976,"['AAACAGAGCGACTCCT-1', 'AAACAGGGTCTATATT-1', '...",551.0
16,ENSG00000188290,"['AAACAAGTATCTCCCA-1', 'AAACACCAATAACTGC-1', '...",781.0
17,ENSG00000187608,"['AAACAAGTATCTCCCA-1', 'AAACCGGGTAGGTACC-1', '...",1058.0
18,ENSG00000188157,"['AAACAAGTATCTCCCA-1', 'AAACCCGAACGAAATC-1', '...",647.0
26,ENSG00000078808,"['AAACAGAGCGACTCCT-1', 'AAACCGTTCGTCCAGG-1', '...",953.0
...,...,...,...
23449,ENSG00000198886,"['AAACAAGTATCTCCCA-1', 'AAACACCAATAACTGC-1', '...",4015.0
23450,ENSG00000198786,"['AAACAAGTATCTCCCA-1', 'AAACACCAATAACTGC-1', '...",3877.0
23451,ENSG00000198695,"['AAACCGGAAATGTTAA-1', 'AAACCGGGTAGGTACC-1', '...",1690.0
23452,ENSG00000198727,"['AAACAAGTATCTCCCA-1', 'AAACACCAATAACTGC-1', '...",4015.0


In [548]:
df_507_min

,0,coord_list,coord_count
11,ENSG00000188976,"['AAACCTAAGCAGCCGG-1', 'AAACGAAGAACATACC-1', '...",435.0
16,ENSG00000188290,"['AAACATTTCCCGGATT-1', 'AAACGGGCGTACGGGT-1', '...",709.0
17,ENSG00000187608,"['AAACAACGAATAGTTC-1', 'AAACAAGTATCTCCCA-1', '...",1064.0
18,ENSG00000188157,"['AAACATTTCCCGGATT-1', 'AAACGGGCGTACGGGT-1', '...",534.0
26,ENSG00000078808,"['AAACAGGGTCTATATT-1', 'AAACCACTACACAGAT-1', '...",756.0
...,...,...,...
23449,ENSG00000198886,"['AAACAACGAATAGTTC-1', 'AAACAAGTATCTCCCA-1', '...",4226.0
23450,ENSG00000198786,"['AAACAAGTATCTCCCA-1', 'AAACAATCTACTAGCA-1', '...",4149.0
23451,ENSG00000198695,"['AAACACCAATAACTGC-1', 'AAACAGGGTCTATATT-1', '...",1242.0
23452,ENSG00000198727,"['AAACAACGAATAGTTC-1', 'AAACAAGTATCTCCCA-1', '...",4224.0


In [549]:
df_507_res = df_507_min.reset_index()
df_507_res

,index,0,coord_list,coord_count
0,11,ENSG00000188976,"['AAACCTAAGCAGCCGG-1', 'AAACGAAGAACATACC-1', '...",435.0
1,16,ENSG00000188290,"['AAACATTTCCCGGATT-1', 'AAACGGGCGTACGGGT-1', '...",709.0
2,17,ENSG00000187608,"['AAACAACGAATAGTTC-1', 'AAACAAGTATCTCCCA-1', '...",1064.0
3,18,ENSG00000188157,"['AAACATTTCCCGGATT-1', 'AAACGGGCGTACGGGT-1', '...",534.0
4,26,ENSG00000078808,"['AAACAGGGTCTATATT-1', 'AAACCACTACACAGAT-1', '...",756.0
...,...,...,...,...
6096,23449,ENSG00000198886,"['AAACAACGAATAGTTC-1', 'AAACAAGTATCTCCCA-1', '...",4226.0
6097,23450,ENSG00000198786,"['AAACAAGTATCTCCCA-1', 'AAACAATCTACTAGCA-1', '...",4149.0
6098,23451,ENSG00000198695,"['AAACACCAATAACTGC-1', 'AAACAGGGTCTATATT-1', '...",1242.0
6099,23452,ENSG00000198727,"['AAACAACGAATAGTTC-1', 'AAACAAGTATCTCCCA-1', '...",4224.0


In [550]:
max_row_indices = df_507_res['coord_count'].idxmax()
max_row_indices

6089

In [451]:
df_res.loc[max_row_indices]

index                                                      23440
0                                                ENSG00000198888
coord_list     ['AAACAAGTATCTCCCA-1', 'AAACAATCTACTAGCA-1', '...
coord_count                                               3639.0
Name: 8090, dtype: object

In [525]:
df_672_res.loc[0,0]

'ENSG00000188976'

<h5> from coordinate list into 2D matrix

In [551]:
# for i in range(len(df_res)):

directory = 'datapoints/151507/'

for i in range(len(df_507_res)):

    # get coord_list from dataframe as string back to list
    my_list = ast.literal_eval(df_507_res.loc[i,'coord_list'])

    # table for converting coordinate list into actual pixel coordinates
    ext_coord = coord507.loc[my_list]

    # points = coord673.values.tolist()
    # sites = coord673.index.tolist()
    points = ext_coord.values.tolist()
    sites = ext_coord.index.tolist()
    
    x, y = zip(*points)
    
    # Convert into 2D matrix
    x_resolution = 200
    y_resolution = 250

    x_min = min(x)
    x_max = max(x)
    x_ptp = x_max - x_min

    y_min = min(y)
    y_max = max(y)
    y_ptp = y_max - y_min

    img = np.zeros((y_resolution+20, x_resolution+20), dtype=bool)

    for k, l in points:
        x_pxl = (k - x_min) / x_ptp * x_resolution
        y_pxl = (l - y_min) / y_ptp * y_resolution
        img[int(y_pxl+10), int(x_pxl+10)] = 1
    
    np.save(directory+df_507_res.loc[i,0]+'.npy',img)

In [465]:
# for i in range(len(points)):
#     for j in range(len(points[i])):
#         points[i][j] = points[i][j] * 0.02
# points[0:4]

In [466]:
# x, y = zip(*points)

In [467]:
# print(points[0],x[0],y[0])

[9791.0, 8468.0] 9791.0 8468.0


In [552]:
# for i, filename in enumerate(os.listdir('datapoints/151673')):
#     print(filename)
directory = 'datapoints/151507'
len(os.listdir(directory))

6101

In [553]:
filenames = os.listdir(directory)
filenames[:3]

['ENSG00000137168.npy', 'ENSG00000139168.npy', 'ENSG00000072518.npy']

In [ ]:
plt.figure(figsize=(8,10))

fig, ax = plt.subplots(3,1,figsize=(12,36))

for i, filename in enumerate(filenames[:3]) :
    if filename.endswith('.npy') and os.path.isfile(os.path.join(directory, filename)):
        new_test = np.load(directory+'/'+filename)   
        ax[i].imshow(new_test, cmap='viridis',origin='lower')

plt.show()

In [ ]:
# 2D matrix are formed, but seems to be too sparse to be learned need to rescale
plt.figure(figsize=(8,10))

fig, ax = plt.subplots(figsize=(8,10))
ax.imshow(img, cmap='viridis',origin='lower')
plt.savefig('plot.png')
plt.show()

In [ ]:
# 2D matrix are formed, but seems to be too sparse to be learned need to rescale
plt.figure(figsize=(8,10))

fig, ax = plt.subplots(figsize=(8,10))
ax.imshow(img, cmap='viridis',origin='lower')
# plt.savefig('plot.png')
plt.show()

In [ ]:
plt.figure(figsize=(8,10))
plt.scatter(x,y,s=5,c='b')
plt.show()

In [202]:
len(sites)

3639

<h1>Feed to NN

In [ ]:
# NN take matrix of gene distribution

# make into tensor

# run NN model

# output should be decoder part of hidden features

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"
train_root_path = '/home/Spatial_transcriptomic/VERIFY/train' 
test_root_path = '/home/Spatial_transcriptomic/VERIFY/test'
batch_size = 300
epochs = 200
learning_rate = 1e-3
im_size = size_10
# torch.manual_seed(42)

In [ ]:
instance_X_train, instance_X_test = train_test_split(instance_imgs_10, test_size=0.20, random_state=42,shuffle=True) 


train_dataset = TumorDatasetTrain(instance=instance_X_train, root_dir=train_root_path,
                                transform=transforms.Compose([ToTensor()]))
test_dataset = TumorDatasetTest(instance=instance_X_test, root_dir=test_root_path, 
                                transform=transforms.Compose([ToTensor()]))
all_dataset = TumorDatasetAll(instance=instance_imgs_10, root_dir=train_root_path,
                                transform=transforms.Compose([ToTensor()]))

dataloader_all = DataLoader(all_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
dataloader_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
dataloader_test = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=1)

running_loss = []
running_vloss = []

In [ ]:
model_path1 = "/home/Spatial_transcriptomic/VERIFY/lab06_AE_weight_decay.pth"
save = False
pret = False

In [ ]:
#  use gpu if available
start_time1 = time.time()

device = torch.device("cuda")

# create a model from `AE` autoencoder class
# load it to the specified device, either gpu or cpu
# model = ConvAutoencoder().to(device)
model = AE(input_shape=im_size*im_size*33).to(device)
model.double()

# create an optimizer object
# Adam optimizer with learning rate 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# mean-squared error loss
criterion = nn.MSELoss()
running_loss = []
running_vloss = []

#     Auto Encoder Training
print('Training AE')
if (not os.path.exists(model_path1) or not pret):
    for epoch in range(epochs):
        start_time = time.time()
        loss = 0
        vloss = 0
        model.train()
        for batch_ndx, batch_features in enumerate(dataloader_train):
            # reshape mini-batch data to [N, 784] matrix
            # load it to the active device
    #             batch_features = Variable(batch_features['image']).to(device)
    #             batch_features = batch_features.type(torch.cuda.FloatTensor)
            images = Variable(batch_features['image']).view(-1,im_size*im_size*33).to(device)
    #             images = images.type(torch.cuda.FloatTensor)
            # reset the gradients back to zero
            # PyTorch accumulates gradients on subsequent backward passes
            optimizer.zero_grad()

            # compute reconstructions
            outputs, coded = model(images)
#             print('size', outputs.size(), coded.size())
    #             encoder_only_nosig = model.encoder(images)
    #             encoder_only = torch.sigmoid(model.encoder(images))
            # compute training reconstruction loss
            train_loss = criterion(outputs, images)

            # compute accumulated gradients
            train_loss.backward()

            # perform parameter update based on current gradients
            optimizer.step()
            # add the mini-batch training loss to epoch loss
            loss += train_loss.item()

        # compute the epoch training loss
        loss = loss / len(dataloader_train)
        running_loss.append(loss)
        # display the epoch training loss
        #if(epoch % 5 == 0):
    #       ------------------------------------------------------------------------  
        model.eval()
        with torch.no_grad():
            for batch_ndx, batch_features in enumerate(dataloader_test):
                # reshape mini-batch data to [N, 784] matrix
                # load it to the active device
    #             batch_features = Variable(batch_features['image']).to(device)
    #             batch_features = batch_features.type(torch.cuda.FloatTensor)
                images = Variable(batch_features['image']).view(-1,im_size*im_size*33).to(device)
    #                 images = images.type(torch.cuda.FloatTensor)
                # reset the gradients back to zero
                # PyTorch accumulates gradients on subsequent backward passes
                optimizer.zero_grad()

                # compute reconstructions
                outputs, coded = model(images)
    #             encoder_only_nosig = model.encoder(images)
    #             encoder_only = torch.sigmoid(model.encoder(images))
                # compute training reconstruction loss
                valid_loss = criterion(outputs, images)

                # perform parameter update based on current gradients
    #                 optimizer.step()
                # add the mini-batch training loss to epoch loss
                vloss += valid_loss.item()

            # compute the epoch training loss
            vloss = vloss / len(dataloader_test)
            running_vloss.append(vloss)

    #     -----------------------------------------------------------------------------
        current_time = time.time()
        elapsed_time = current_time - start_time
        print("epoch : {}/{}, recon loss = {:.8f}, valid loss = {:.8f}, time = {:.2f}s".format(epoch + 1, epochs, loss, vloss, elapsed_time))


        if (epoch > 3) and (abs(running_vloss[epoch] - running_vloss[epoch-1]) <= 0.0001) and (abs(running_vloss[epoch - 1] - running_vloss[epoch - 2]) <= 0.0001):
            print('satisfied early stop')
            break
    if save:
        torch.save(model.state_dict(), model_path1)

In [123]:
coord_list = [[1.2, 3.4], [2.3, 4.5], [3.4, 5.6], [4.5, 6.7]]

# convert the list to a PyTorch tensor
coords = torch.tensor(coord_list)

# reshape the tensor to the required input shape (batch size, number of channels, height, width)
coords = coords.view(1, 1, -1, 2)

In [124]:
coords

tensor([[[[1.2000, 3.4000],
          [2.3000, 4.5000],
          [3.4000, 5.6000],
          [4.5000, 6.7000]]]])

<h1>Run BEER

In [ ]:
# get decoder hidden feature and concatenate with assays

# final matrix includes ~100 extra features of spatial embeddings treated as spots

# BEER should results in a Umap plot, Correlation plot, and no-Batch effect umap